In [2]:
# CIFAR-10 Photo Classification Dataset

# The dataset is comprised of 60,000 32×32 pixel color photographs of objects from 10 classes, such as frogs, birds, cats,ect .The class labels and their standard associated integer values are listed below.

# 0: airplane
# 1: automobile
# 2: bird
# 3: cat
# 4: deer
# 5: dog
# 6: frog
# 7: horse
# 8: ship
# 9: truck


In [5]:
# In short, torch.device in PyTorch is used for specifying where tensors should be allocated 
# and operations executed—whether on CPU or GPU.

device=th.device("cuda" if th.cuda.is_available() else "cpu")

In [4]:
# required pytorch labarires
 
import torch as th
import torch.nn as nn
import torch.optim as op
import torchvision
import torchvision.datasets as dsets

In [6]:
# In our specific case, we're using torchvision.transforms.ToTensor(), which converts PIL images or
#  numpy arrays to PyTorch tensors.

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

In [7]:
# The following is a program to do image classification on the CIFAR-10 dataset

train_dset = dsets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dset = dsets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
train_dset,test_dset

(Dataset CIFAR10
     Number of datapoints: 50000
     Root location: ./data
     Split: Train
     StandardTransform
 Transform: Compose(
                ToTensor()
            ),
 Dataset CIFAR10
     Number of datapoints: 10000
     Root location: ./data
     Split: Test
     StandardTransform
 Transform: Compose(
                ToTensor()
            ))

In [9]:
train_dset.data.shape

(50000, 32, 32, 3)

In [10]:
test_dset.data.shape

(10000, 32, 32, 3)

In [11]:
# class of extracted dataset
train_dset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [12]:
#classes: A list containing the names of each class.

classes=['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [13]:
# torch.utils.data.DataLodaer uses that allow us to use pre-loaded datasets as well as your own data

train_loader=th.utils.data.DataLoader(dataset=train_dset,batch_size=4,shuffle=True)

test_loader=th.utils.data.DataLoader(dataset=test_dset,batch_size=4,shuffle=False)

In [14]:
model = nn.Sequential(     
    nn.Conv2d(3, 6, 5),#Defines the first convolutional layer with 3 input channels, 6 output channels, and a kernel size of 5x5.
    
    nn.ReLU(),#Applies the Rectified Linear Unit (ReLU) activation function element-wise to introduce non-linearity after each convolutional layer.
    
    nn.MaxPool2d(2, 2),#Performs 2x2 max pooling operation with a stride of 2, reducing the spatial dimensions of the input by a factor of 2.
    
    nn.Conv2d(6, 16, 5),#Defines the second convolutional layer with 6 input channels (from the previous layer), 16 output channels, and a kernel size of 5x5.
    nn.ReLU(),
    
    nn.MaxPool2d(2, 2),#Another max pooling operation, reducing  dimensions further.
    
    nn.Flatten(),#Flattens the multi-dimensional input tensor into a one-dimensional tensor, preparing it for fully connected layers.
    
    nn.Linear(16 * 5 * 5, 120), #Defines a fully connected layer with 16 * 5 * 5 input features (output of the previous layers), 120 output features.
    nn.ReLU(),
    nn.Linear(120, 84),#Another fully connected layer with 120 input features and 84 output features.
    nn.ReLU(),
    nn.Linear(84, 10) #Final fully connected layer with 84 input features and 10 output features, corresponding to the number of classes in the classification task.
).to(device)# to device at the end indicates that the model will be moved to the specified deivce(eg "CPU"or"GPU")for computation 

In [15]:
model

Sequential(
  (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=400, out_features=120, bias=True)
  (8): ReLU()
  (9): Linear(in_features=120, out_features=84, bias=True)
  (10): ReLU()
  (11): Linear(in_features=84, out_features=10, bias=True)
)

In [16]:
# traing the model

criterion=nn.CrossEntropyLoss()
optimizer=op.Adam(model.parameters(),lr=0.001)

In [32]:
num_epochs=5
n_total_steps=len(train_loader)
for epoch in range(num_epochs):
    for i ,(images,label) in enumerate(train_loader):

        image=images.to(device)
        labels=label.to(device)
        
        
        #forward pass
        output=model(image)
        loss=criterion(output,label)
        
        
        #backward and optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        #Printing Training Progress:
        if (i+1) % 2000 ==0:
            print(f"Epoch [{epoch+1}/{num_epochs}],step [{i+1}/{n_total_steps}],Loss : {loss.item() : 4f}")

            
# Saving the Trained Model:         
print('Finished Training')
PATH = './cnn.pth'
th.save(model.state_dict(), PATH) # to save the model

Epoch [1/5],step [2000/12500],Loss :  1.628963
Epoch [1/5],step [4000/12500],Loss :  1.446648
Epoch [1/5],step [6000/12500],Loss :  1.174452
Epoch [1/5],step [8000/12500],Loss :  3.066168
Epoch [1/5],step [10000/12500],Loss :  0.996582
Epoch [1/5],step [12000/12500],Loss :  2.395263
Epoch [2/5],step [2000/12500],Loss :  2.083189
Epoch [2/5],step [4000/12500],Loss :  0.896975
Epoch [2/5],step [6000/12500],Loss :  0.883373
Epoch [2/5],step [8000/12500],Loss :  1.390809
Epoch [2/5],step [10000/12500],Loss :  2.310724
Epoch [2/5],step [12000/12500],Loss :  1.694226
Epoch [3/5],step [2000/12500],Loss :  0.734928
Epoch [3/5],step [4000/12500],Loss :  1.009854
Epoch [3/5],step [6000/12500],Loss :  1.115263
Epoch [3/5],step [8000/12500],Loss :  1.180428
Epoch [3/5],step [10000/12500],Loss :  2.501098
Epoch [3/5],step [12000/12500],Loss :  1.582311
Epoch [4/5],step [2000/12500],Loss :  0.795771
Epoch [4/5],step [4000/12500],Loss :  0.523436
Epoch [4/5],step [6000/12500],Loss :  1.824321
Epoch [

In [47]:
with th.no_grad():
    n_correct = 0 #n_correct: The number of correctly classified samples.
    n_samples = 0 #n_samples: The total number of samples.
    n_class_correct = [0 for i in range(10)] #n_class_correct: A list or array containing the number of correctly classified samples for each class.
    n_class_samples = [0 for i in range(10)] #n_class_samples: A list or array containing the total number of samples for each class.
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        # max returns (value ,index)
        _, predicted = th.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(4):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %') #classes: A list containing the names of each class.

Accuracy of the network: 56.13 %
Accuracy of airplane: 69.3 %
Accuracy of automobile: 69.0 %
Accuracy of bird: 37.7 %
Accuracy of cat: 40.3 %
Accuracy of deer: 46.4 %
Accuracy of dog: 46.9 %
Accuracy of frog: 65.4 %
Accuracy of horse: 52.4 %
Accuracy of ship: 72.3 %
Accuracy of truck: 61.6 %
